# GONZAG WestMed example

This notebook demonstrates how Gonzag can be used to:
- perform a space-time interpolation of SSH on a gridded OGCM domain onto a satellite track (provided in 2 NetCDF input files)
- select and retain N valid SSH along track segments (both for input satellite SSH and interpolated model SSH)
- preprocess these N SSH segments prior to applying a Fast Fourrier Transform (FFT)
- apply the FFT
- plot the the mean spectrum (of the N spectra)

## Loading & initializations

In [1]:
import sys
import os
import glob
from os import getenv
import warnings
warnings.filterwarnings("ignore")

import numpy as nmp
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

GONZAG_DIR   = '/Users/auraoupa/Work/git/gonzag_cloud'
sys.path.append(GONZAG_DIR)

import gonzag as gz

GONZAG_DATA_DIR = '/Users/auraoupa/Data/gonzag/gonzag_input'

# Satellite input data:
name_sat = 'Sentinel3A'
file_sat = GONZAG_DATA_DIR+'/SENTINEL3A_20170130-20170303.nc'
name_ssh_sat = 'sla_unfiltered'
name_time_sat='time'
period_sat=['2017-02-01','2017-02-28'] #period of satellite is bigger than model so we restrict to model period

# Model input data:
name_mod = 'eNATL60-WestMed'
file_mod = GONZAG_DATA_DIR+'/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc'
name_ssh_mod = 'sossheig'
name_lat_mod = 'gphit'
name_lon_mod = 'glamt'
name_time_mod = 'time_counter'
file_lsm_mod = file_mod; name_lsm_mod = '_FillValue' ; # we use _FillValue attribute of "nams_ssh_mod" in "file_mod"
clsm = name_lsm_mod
if name_lsm_mod=='_FillValue': clsm = name_lsm_mod+'@'+name_ssh_mod

period_mod=['2017-02-01','2017-02-28']
l_griddist = False ; # grid is not strongly distorded





In [2]:
import xarray as xr
fresults=sorted(glob.glob('../results/results_'+name_sat+'-'+name_mod+'/result_??.nc'))
dsn=xr.open_mfdataset(fresults,concat_dim='time',combine='nested')
dsn

<xarray.Dataset>
Dimensions:         (time: 2048)
Coordinates:
  * time            (time) float64 1.486e+18 1.486e+18 ... 1.486e+18 1.486e+18
Data variables:
    latitude        (time) float64 dask.array<chunksize=(110,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(110,), meta=np.ndarray>
    sossheig_np     (time) float64 dask.array<chunksize=(110,), meta=np.ndarray>
    sossheig_bl     (time) float64 dask.array<chunksize=(110,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(110,), meta=np.ndarray>
    distance        (time) float64 dask.array<chunksize=(110,), meta=np.ndarray>
    time_counter    (time) datetime64[ns] dask.array<chunksize=(110,), meta=np.ndarray>
Attributes:
    about:    Created with Gonzag package => https://github.com/brodeau/gonzag

In [3]:
ds0=xr.open_dataset('../results/results_'+name_sat+'-'+name_mod+'/result_00.nc')

In [7]:
ds0

<xarray.Dataset>
Dimensions:         (time: 110)
Coordinates:
  * time            (time) float64 1.486e+18 1.486e+18 ... 1.486e+18 1.486e+18
Data variables:
    latitude        (time) float64 36.85 36.91 36.96 37.02 ... 43.1 43.16 43.21
    longitude       (time) float64 5.552 5.534 5.516 5.498 ... 3.482 3.461 3.44
    sossheig_np     (time) float64 -0.4036 -0.4197 -0.4445 ... -0.5779 -0.5742
    sossheig_bl     (time) float64 -0.4042 -0.4191 -0.4471 ... -0.5776 -0.574
    sla_unfiltered  (time) float64 0.16 0.144 0.16 0.129 ... 0.096 0.081 0.085
    distance        (time) float64 0.0 6.691 13.38 ... 1.128e+03 1.134e+03
    time_counter    (time) datetime64[ns] 2017-02-05T21:11:44.566244096 ... 2...
Attributes:
    about:    Created with Gonzag package => https://github.com/brodeau/gonzag

In [8]:
ds0.sossheig_np


<xarray.DataArray 'sossheig_np' (time: 110)>
array([-0.403595, -0.419703, -0.444466, -0.481346, -0.500922, -0.517359,
       -0.539108, -0.553277, -0.571385, -0.57922 , -0.595365, -0.607116,
       -0.612966, -0.618696, -0.623253, -0.613938, -0.603765, -0.598771,
       -0.590283, -0.576692, -0.564747, -0.5513  , -0.543447, -0.538448,
       -0.534739, -0.540753, -0.544268, -0.529713, -0.525829, -0.521405,
       -0.521824, -0.520739, -0.522594, -0.530561, -0.547299, -0.565047,
       -0.583759, -0.601935, -0.623002, -0.631634, -0.622311, -0.613148,
       -0.608969, -0.609825, -0.61883 , -0.638391, -0.632012, -0.624939,
       -0.619575, -0.621279, -0.631898, -0.644042, -0.640623, -0.643296,
       -0.653976, -0.661158, -0.662614, -0.657872, -0.661885, -0.662451,
       -0.653583, -0.65016 , -0.645059, -0.622413, -0.607095, -0.588575,
       -0.563161, -0.546799, -0.53248 , -0.517596, -0.506103, -0.50737 ,
       -0.516104, -0.529374, -0.544789, -0.567934, -0.59562 , -0.607301,
       -0.620448, -0.634232, -0.639935, -0.639124, -0.639495, -0.647855,
       -0.641804, -0.643498, -0.65179 , -0.656404, -0.654265, -0.650817,
       -0.647145, -0.639556, -0.634758, -0.609577, -0.591198, -0.578386,
       -0.577569, -0.579743, -0.582047, -0.572253, -0.5639  , -0.562082,
       -0.567201, -0.572048, -0.576616, -0.581773, -0.5831  , -0.580249,
       -0.57788 , -0.574226])
Coordinates:
  * time     (time) float64 1.486e+18 1.486e+18 ... 1.486e+18 1.486e+18
Attributes:
    units:      m
    long_name:  Model SSH interpolated in space (nearest-point) and time on s...

In [ ]:
dsn.sossheig_np.plot()

### 2] Plot time series of _interpolated model_ SLA and satellite SLA

Even if probably ugly it gives the pictures of what we are dealing with...

In [ ]:
import matplotlib.dates as mdates
                                                                                                        
clr_sat = '#AD0000'
clr_mod = '#008ab8'

VT = dsn.time ; # time from UNIX Epoch to Matlplotlib friendly...

fig = plt.figure(num = 1, figsize=(12,7), facecolor='w', edgecolor='k')
ax = plt.axes([0.07, 0.24, 0.9, 0.75])
ax.set_xticks(VT[::200])
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.xticks(rotation='60')
plt.plot(VT, VT*0.0          , '-', color='k',                   label=None,  zorder=5)
plt.plot(VT, dsn.sla_unfiltered-nmp.mean(dsn.sla_unfiltered), '.', color=clr_sat, markersize=5, alpha=0.5, label=name_sat, zorder=10)
plt.plot(VT, dsn.sossheig_np-nmp.mean(dsn.sossheig_np), '.', color=clr_mod, markersize=5, alpha=0.5, label=name_mod, zorder=15)
#ax.set_ylim(-r_max_amp_ssh,r_max_amp_ssh) ;
#ax.set_xlim(VT[0],VT[-1])
plt.ylabel('SSH [m]')
ax.grid(color='k', linestyle='-', linewidth=0.3)
lgnd = plt.legend(bbox_to_anchor=(0.55, 1.), ncol=1, shadow=True, fancybox=True)


## Processing solution for spectral analysis
### 1] Select continuous segments, prepare them for FFT, and apply FFT on each of them

In [ ]:
# Extract the Ns continuous data segments:                                                                                                              
ISeg_beg, ISeg_end = gzg.FindUnbrokenSegments( Solution.time, Solution.distance, Solution.ssh_mod, \
                                             rcut_time=1.2, rcut_dist=7.8 )
# => any Δt > 1.2 s between 2 consecutive points is considered as a gap and hence a cut!
# => any Δd > 7.8 km    "               "             "                 "

# Select and retain only the "NbSeg" proper segments:                                                                                                               
NbSeg, Nsl, IDEDSeg = gzg.SegmentSelection(ISeg_beg, ISeg_end, np_valid_seg=70)
# Debug validity check:                                                                                                                                       
#for js in range(NbSeg):                                                                                                                                
#    print(' * Seg # ',js+1,' => it1, it2 =', IDEDSeg[js,:], ' ==> len = ', IDEDSeg[js,1]-IDEDSeg[js,0]+1)                                              
#print(' Nsl = ',Nsl)

# Process data on segment so ready for FFT:                                                                                                 
XPs, XPm, rdist_sample = gzg.Process4FFT( IDEDSeg, Solution.distance, Solution.ssh_mod, Solution.ssh_sat )

# Apply FFT !                                                                                                                               
Kwn, PwSpc_s, PwSpc_m = gzg.ApplyFFT( IDEDSeg, XPs, XPm, rdist_sample )


### 2] Plotting the mean spectrum

In [ ]:
CLIMPORN_DIR = '/Users/auraoupa/Work/git/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp

# Building our spectrum as the mean of the NbSeg spectra:
vps_mod = nmp.mean(PwSpc_m[:,:],axis=0)
vps_sat = nmp.mean(PwSpc_s[:,:],axis=0)

# Blabla for the plot:
cinfrm = str(NbSeg)+' segments\n'+str(Nsl)+' points/segment\n'+r'$\Delta$d sat.: '+str(round(rdist_sample,1))+' km'

ii = cp.plot("pow_spectrum_ssh")(Kwn, vps_mod, clab1=name_mod+' ("'+name_ssh_mod+'")', clr1=clr_mod, lw1=5, \
                                 cinfo=cinfrm, logo_on=False, \
                                 L_min=13., L_max=500., P_min_y=-6, P_max_y=1, \
                                 l_show_k4=False, l_show_k5=True, l_show_k11o3=False, l_show_k2=True, \
                                 vk2=Kwn, vps2=vps_sat, clab2=name_sat+' ("'+name_ssh_sat+'")', clr2=clr_sat, lw2=4)
